# Loop Guards & Moments After Termination

Polar can analyze stochastic processes that do not stop given by a probabilistic loop with the loop guard `true`.
However, Polar can also analyze loops with a proper guard. Allowing for any loop guard quickly gets us in the real of undecidability and uncomputable problems.
But, Polar guarantees that for the loops it accepts it will theoretically be able to give an answer. That is why not any loop guard is allowed.
The restrictions on the loop guard are the same as the ones for the if-conditions: *all variables in the loop guard must only assume finitely many values*.
In fact, when Polar converts the input loop into its normal form, the loop guard is converted into an if-statement that wraps to whole loop body. Let's look at an example:

In [2]:
!cat loops/geometric.prob


stop = 0
steps = 0
x = 1
while stop == 0:
    stop = Bernoulli(1/2)
    x = 2*x
    steps = steps + 1
end


The variable `stop` in the guard can only assume the values `0` and `1`. Hence, the guard is supported by Polar.
We can still ask Polar to compute closed-form formulas for moments of program variables parameterized by the number of loop iterations `n`.
An important question is: "What is the meaning of this formula if the loop terminates in less than `n` many iterations?".
As mentioned, Polar converts a guarded loop into an unguarded loop by wrapping the loop body into an if-statement for which the condition is the original loop guard.
Hence, every variable still corresponds to an infinite sequence of numbers. However, after the loop guard is falsified (and the loop would terminate), the values of the program variables stay the same. Let's compute some closed-form formulas for the exepcted values of the program variables:

In [6]:
!python ../polar.py loops/geometric.prob --goals "E(stop)" "E(steps)" "E(x)"


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

E(stop) = 0; 1 - 1/2**n
Solution is exact

E(steps) = 0; 2 - 2/2**n
Solution is exact

E(x) = 1; 2; n + 1
Solution is exact

Elapsed time: 0.42545104026794434 s


The closed-form for `E(stop)` is `1 - 1/2**n`. Which can intuitively be explained as follows: once stop becomes `1` it will remain `1` forever. Moreover, in every iteration there is a fifty-fifty chance of `stop` becoming `1`.

Polar also provides functionality to compute the moments of program variables after termination by using the `after_loop` parameter:

In [1]:
!python ../polar.py loops/geometric.prob --goals "E(stop)" "E(steps)" "E(x)" --after_loop


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

E(stop) = 1
Solution is exact

E(steps) = 2
Solution is exact

E(x) = oo
Solution is exact

Elapsed time: 1.2863147258758545 s


We can see that the expected value of `stop` is `1` after termination. In fact, `stop` will be `1` with absolute certainty, because the loop terminates precisely when `stop` is `1`. Moreover, the loop has an expected runtime of `2` and surprisingly the expected value of `x` after termination is infinite. This phenomenon can be intuitively explained as follows: later iterations become more and more unlikely. The chance of not terminating decreases exponentially. However, the value of `x` increases exponentially. The exponential decrease of the probability of non-termination and the exponential increase of `x` cancel each other out.

Another possibility the get some information after the loop terminates is to compute the invariant ideal ([see the notebook on this topic](invariants.ipynb)).
The invariants Polar computes are equations among the moments of program variables that are true after every iteration of the loop. Because of our mechanism of converting guarded loops into unguarded loops by using an if-statement the computed invariants are also true after termination of the loop. Let's see what we get for our example:

In [3]:
!python ../polar.py loops/geometric.prob --goals "E(stop)" "E(steps)" "E(x)" --invariants


8888888b.   .d88888b.  888             d8888 8888888b.
888   Y88b d88P" "Y88b 888            d88888 888   Y88b
888    888 888     888 888           d88P888 888    888
888   d88P 888     888 888          d88P 888 888   d88P
8888888P"  888     888 888         d88P  888 8888888P"
888        888     888 888        d88P   888 888 T88b
888        Y88b. .d88P 888       d8888888888 888  T88b
888         "Y88888P"  88888888 d88P     888 888   T88b

By the ProbInG group



-------------------
- Analysis Result -
-------------------

E(stop) = 0; 1 - 1/2**n
Solution is exact

E(steps) = 0; 2 - 2/2**n
Solution is exact

E(x) = 1; 2; n + 1
Solution is exact


-------------------
-   Invariants    -
-------------------

Following is a gröbner basis for the invariant ideal:

-E(steps) + 2*E(stop) = 0

Elapsed time: 0.45613980293273926 s


Rewritten, the invariant we get states `E(steps) = 2*E(stop)`. We know from the previous output that after termination `E(steps) = 2` and `E(stop) = 1` which is consistent with the invariant we got.

We can compute the basis for the invariant ideal also using Python code as follows:

In [5]:
from inputparser import Parser
from program import normalize_program
from recurrences import RecBuilder
from recurrences.solver import RecurrenceSolver
from invariants import InvariantIdeal

program = Parser().parse_file("loops/geometric.prob")
# Construct normal form so that Polar can analyze it
program = normalize_program(program)

# Construct and solve recurrences
rec_builder = RecBuilder(program)
monomials = ["steps", "stop", "x"]
closed_forms = {}
for monomial in monomials:
    # Construct the recurrences describing E(monomial) -> expected value of monomial
    recurrences = rec_builder.get_recurrences(monomial)
    # solve and save the closed-forms (use E(monomial) as the id because the loop is probabilistic)
    closed_forms[f"E({monomial})"] = RecurrenceSolver(recurrences).get(monomial)

# Construct the invariant ideal
invariant_ideal = InvariantIdeal(closed_forms)
basis = invariant_ideal.compute_basis()
print(basis)

{E(steps) - 2*E(stop)}
